<a href="https://colab.research.google.com/github/samitha278/miniVGG/blob/main/build_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Simple Convolution function

In [49]:
def Convolution(image,kernel):

  x,y = image.shape

  a,b = kernel.shape

  feature_map = []

  for i in range(y-b):

    row = []

    for j in range(x-a):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [50]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel)


In [51]:
feature_map.shape

torch.Size([29, 29])

## Convolution with stride

In [52]:
def Convolution(image,kernel,stride=(1,1)):

  x,y = image.shape
  a,b = kernel.shape

  r,c = stride

  feature_map = []

  for i in range(0,y-b,c):

    row = []

    for j in range(0,x-a,r):

      temp = (image[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [53]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel,(2,1))      # 2: aloge side rows , 1: along side columns


In [54]:
feature_map.shape

torch.Size([29, 15])

## Adding Padding

In [55]:
def add_padding(image, padding):



  r,c = padding

  if r>0 :
    rows = torch.zeros((r,image.shape[1]))

    image = torch.cat((rows , image , rows),dim=0)

  if c>0:

    columns = torch.zeros((c,image.shape[0]))

    image = torch.cat((columns , image.T , columns),dim=0)


  return image.T



In [56]:
add_padding(torch.randn(2,2),(2,3))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.8457, -0.2871,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.2280,  0.6533,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [57]:
def Convolution(image,kernel,stride=(1,1),padding = (0,0)):


  a,b = kernel.shape

  r,c = stride

  image_pd = image if padding==(0,0) else add_padding(image,padding)

  feature_map = []

  x,y = image_pd.shape

  for i in range(0,y-b,c):

    row = []

    for j in range(0,x-a,r):

      temp = (image_pd[i:i+b,j:j+a] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [58]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel, padding=(2,2))


In [59]:
feature_map.shape

torch.Size([33, 33])

### Simple adding pad func

In [60]:
def add_padding2(matrix,padding):

  n,m = matrix.shape

  r,c = padding

  padded_matrix = torch.zeros((n+r*2,m+c*2))

  padded_matrix[r:r+n,c:c+m] = matrix

  return padded_matrix

In [61]:
add_padding2(torch.randn(2,2),(2,3))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, -0.9070, -0.0018,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.8128, -1.2010,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

### Adding Dialation

In [62]:
def Convolution(image,kernel,stride=(1,1),padding = (0,0),dilation=(1,1)):


  a,b = kernel.shape

  r,c = stride

  d1,d2 = dilation

  image_pd = image if padding==(0,0) else add_padding2(image,padding)

  feature_map = []

  x,y = image_pd.shape


  for i in range(0,x-a*d1+1,r):

    row = []

    for j in range(0,y-b*d2+1,c):

      temp = (image_pd[i:i+a*d1:d1,j:j+b*d2:d2] * kernel).sum(dim=(0,1))

      row.append(temp.item())

    feature_map.append(row)

  return torch.tensor(feature_map)

In [63]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)


feature_map = Convolution(image,kernel,(2,2),(2,2) ,dilation=(2,2))

In [64]:
feature_map.shape

torch.Size([16, 16])

### Test : comparision with pytorch conv2d

In [65]:
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)

out_custom = Convolution(image, kernel, stride=(2,2), padding=(1,1), dilation=(2,2))


img4d = image.unsqueeze(0).unsqueeze(0)       # (1,1,4,4)
kernel4d = kernel.unsqueeze(0).unsqueeze(0) # (1,1,2,2)

out_torch = F.conv2d(img4d, kernel4d, stride=(2,2), padding=(1,1), dilation=(2,2))

print("Custom:\n", out_custom.shape)
print("Torch:\n", out_torch.squeeze().shape)
print("Match? ->", torch.allclose(out_custom,out_torch.squeeze()))

Custom:
 torch.Size([15, 15])
Torch:
 torch.Size([15, 15])
Match? -> True


# All at once

In [66]:
def add_padding2(matrix,padding):
  n,m = matrix.shape
  r,c = padding

  padded_matrix = torch.zeros((n+r*2,m+c*2))
  padded_matrix[r:r+n,c:c+m] = matrix

  return padded_matrix

In [67]:
def Convolution(image, kernel, stride=(1,1), padding=(0,0), dilation=(1,1)):
    a, b = kernel.shape
    r, c = stride
    d1, d2 = dilation

    image_pd = image if padding == (0,0) else add_padding2(image, padding)
    feature_map = []
    x, y = image_pd.shape

    for i in range(0, x - a*d1 + 1, r):
        row = []
        for j in range(0, y - b*d2 + 1, c):

            temp = (image_pd[i:i+a*d1:d1, j:j+b*d2:d2] * kernel).sum(dim=(0,1))
            row.append(temp.item())
        feature_map.append(row)

    return torch.tensor(feature_map)

In [68]:
#torch.manual_seed(220064)
image = torch.randn((32,32))
kernel = torch.ones((3,3)) * (9**-1)

out_custom = Convolution(image, kernel, stride=(2,2), padding=(1,1), dilation=(2,2))


img4d = image.unsqueeze(0).unsqueeze(0)       # (1,1,4,4)
kernel4d = kernel.unsqueeze(0).unsqueeze(0) # (1,1,2,2)

out_torch = F.conv2d(img4d, kernel4d, stride=(2,2), padding=(1,1), dilation=(2,2))

print("Custom:\n", out_custom.shape)
print("Torch:\n", out_torch.squeeze().shape)
print("Match? ->", torch.allclose(out_custom,out_torch.squeeze()))

Custom:
 torch.Size([15, 15])
Torch:
 torch.Size([15, 15])
Match? -> True


### Test nn.Conv2d

In [107]:
torch.manual_seed(200)

c_l = nn.Conv2d(2,4,3)      # in channel = 2, out channel = 4, kernel size = 3x3


sd = c_l.__dict__['_parameters']
wei = sd['weight']
bias = sd['bias']

img = torch.randn(2,12,12)


In [108]:
# for outout channel 1
out = Convolution(img[0],wei[0,0]) + Convolution(img[1],wei[0,1]) + bias[0]

In [109]:
out_real = c_l(img)[0]

In [112]:
torch.allclose(out_real,out)

True

In [115]:
out

tensor([[-1.1716, -0.8489,  0.0087,  0.3687,  0.0612, -0.4664, -0.0916,  0.3448,
         -0.3278,  0.4495],
        [-0.9561,  0.1782,  0.3831, -0.2250, -0.4388, -0.5568, -0.3419,  0.2183,
          0.0076, -0.6042],
        [ 0.0471, -0.4671, -0.2238, -0.1945,  0.2181, -0.4026, -0.5394, -0.6788,
         -0.5775, -0.8300],
        [-0.1201,  0.0371, -0.3132, -0.2208, -0.2247, -0.3121,  0.1181, -0.3183,
          0.2815, -1.5969],
        [ 0.3042,  0.3495, -0.1749,  0.1566, -1.6928, -0.1662,  0.3522, -0.1735,
         -0.7710, -0.8561],
        [ 0.1167,  0.8531,  0.7492,  0.0081, -0.7360, -0.0125, -0.1202, -1.4806,
          0.8252,  0.3005],
        [-0.3631, -0.4048,  1.2756,  0.3087, -1.7028,  0.7630,  0.2855, -1.1258,
          0.5164,  0.3389],
        [-0.2170,  0.4665, -0.0310,  0.3007,  0.1645, -0.0165, -0.6288, -0.9614,
         -0.2822,  0.3014],
        [-1.0514, -0.2228, -0.3419,  0.8720, -0.4499,  0.1352, -0.1190, -0.4014,
          0.8269,  0.5460],
        [ 0.5875,  

## Conv2d Class

In [69]:
class conv2d(nn.Module):

  def __init__(self):
    super().__init__()

    self.kernel
    pass



  def forward(self,matrix,kernel,stride=(1,1),padding=(0,0),dilation=(1,1)):

    pass



